# Trade accounting

This notebook decomposes gross exports into value added categories according to the Borin and Mancini (2019) trade accounting framework. The following table describes each category mathematically and intuitively.

| Category | Formula | Description |
| ---- | ------- | ----------- |
| `DAVAX1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{y}_{sr}$ | Domestic value added (DVA) completed in $s$ and sent to $r$ |
| `DAVAX2` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \mathbf{y}_{rr}$ | DVA in intermediates sent to, completed by, and absorbed in $r$ |
| `REX1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r,s} \mathbf{y}_{ru}$ | DVA in intermediates sent to and completed by $r$ then exported to third economy |
| `REX2` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r,s} \mathbf{A}_{ru} \sum_k \sum_{\ell \neq s,r} \mathbf{B}_{uk} \mathbf{y}_{k\ell}$ | DVA in intermediates sent to and re-exported by $r$ and eventually absorbed in third economy |
| `REX3` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r,s} \mathbf{A}_{ru} \sum_k \mathbf{B}_{uk} \mathbf{y}_{kr}$ | DVA in intermediates sent to and re-exported by $r$ and eventually absorbed in $r$ |
| `REF1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \mathbf{y}_{rs}$ | DVA in intermediates sent to and completed by $r$ then exported to $s$ |
| `REF2` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r} \mathbf{A}_{ru} \sum_k \mathbf{B}_{uk} \mathbf{y}_{ks}$ | DVA in intermediates sent to and re-exported by $r$ and eventually absorbed in $s$ |
| `FVA` | $\sum_{t \neq s} \mathbf{v}_t \mathbf{B}^{(s)}_{ts} \mathbf{e}_{sr}$ | Foreign value added (FVA) in gross exports |
| `PDC1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \sum_{s \neq u} \mathbf{A}_{su} \mathbf{B}_{us} \mathbf{e}_{sr}$ | Pure double counting (PDC) of domestic origin |
| `PDC2` | $\sum_{t \neq s} \mathbf{v}_t \mathbf{B}^{(s)}_{ts} \sum_{s \neq u} \mathbf{A}_{su} \mathbf{B}_{us} \mathbf{e}_{sr}$ | PDC of foreign origin |

Select the MRIO version to load from `data/mrio/` as well as the corresponding output filename to be saved in `data/`.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import duckdb
from functions import subset, asvector, zeroout, diagvec, diagmat, diagrow

### Select MRIO version

In [ ]:
input, output = 'adb-mrio.parquet', 'ta.parquet'
# input, output = 'adb-mrio62.parquet', 'ta62.parquet'
# input, output = 'adb-mrio62-const.parquet', 'ta62-const.parquet'

### Parameters

In [ ]:
sectors = pd.read_excel('../data/raw/sectors.xlsx').drop_duplicates(subset='ind', ignore_index=True)
years = duckdb.sql(f"SELECT DISTINCT t FROM read_parquet('../data/mrio/{input}') ORDER BY t").df()['t']
rows = duckdb.sql(f"SELECT COUNT(*) FROM read_parquet('../data/mrio/{input}')").df()

N = 35                                              # Number of sectors
G = int((rows.iloc[0, 0] / len(years) - 7) / N)     # Number of countries + 1
f = 5                                               # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

## Decompositions

In [ ]:
ta, ta_es, ta_os = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd, Yf = zeroout(Y, inverse=True), zeroout(Y)
    v = np.where(x != 0, va/x, 0)
    A = Z @ np.diag(np.where(x != 0, 1/x, 0))
    Ad, Af = zeroout(A, inverse=True), zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        # Country level

        Exports = np.sum(subset(E, s, -s), axis=0)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = subset(v, s) @ subset(Bnots, s, s)
        VB_FC = subset(v, -s) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        ta_s = pd.DataFrame({
            't': int(year), 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s),
            'breakdown': 'none', 'i': 0, 'i5': 0, 'i15': 0,
            'Exports': Exports,
            'DAVAX1': DAVAX1, 'DAVAX2': DAVAX2,
            'REX1': REX1, 'REX2': REX2, 'REX3': REX3,
            'REF1': REF1, 'REF2': REF2,
            'FVA': FVA, 'PDC1': PDC1, 'PDC2': PDC2
        })
        ta = pd.concat([ta, ta_s], ignore_index=True)

        # Breakdown by export sectors

        Exports = subset(E, s, -s)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = np.diag(subset(v, s) @ subset(Bnots, s, s))
        VB_FC = np.diag(subset(v, -s) @ subset(Bnots, -s, s))
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = np.diag(subset(v, s) @ subset(Bnots, s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)
        PDC2 = np.diag(subset(v, -s) @ subset(Bnots, -s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)

        ta_es_s = pd.DataFrame({
            't': int(year), 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'es',
            'i': np.tile(sectors['ind'], G-1),
            'i5': np.tile(sectors['ind5'], G-1),
            'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1), 'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1), 'REX2': asvector(REX2), 'REX3': asvector(REX3),
            'REF1': asvector(REF1), 'REF2': asvector(REF2),
            'FVA': asvector(FVA), 'PDC1': asvector(PDC1), 'PDC2': asvector(PDC2)
        })
        ta_es = pd.concat([ta_es, ta_es_s], ignore_index=True)

        # Breakdown by origin sectors

        VB_DC = np.diag(subset(v, s)) @ subset(Bnots, s, s)
        VB_FC = diagrow(subset(v, -s)) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        ta_os_s = pd.DataFrame({
            't': int(year), 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'os',
            'i': np.tile(sectors['ind'], G-1), 'i5': np.tile(sectors['ind5'], G-1), 'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1), 'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1), 'REX2': asvector(REX2), 'REX3': asvector(REX3),
            'REF1': asvector(REF1), 'REF2': asvector(REF2),
            'FVA': asvector(FVA), 'PDC1': asvector(PDC1), 'PDC2': asvector(PDC2)
        })
        ta_os = pd.concat([ta_os, ta_os_s], ignore_index=True)

    print(f'{year} done')

ta = pd.concat([ta, ta_es], ignore_index=True)
ta = pd.concat([ta, ta_os], ignore_index=True)
ta.to_parquet(f'../data/{output}', index=False)

### View results

In [ ]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/{output}')").df()